In [ ]:
import json
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from networkx.readwrite import json_graph

%matplotlib inline

In [ ]:
from IngestionModule.twitterlistener import Dummy 

help(Dummy)

In [ ]:
import _generateGraph as gg

help(gg.HashtagGraph())

In [ ]:
import _loadData as l

help(l.LoadData())

### Loading up the HTT dataset

In [ ]:
# Loading and Specifying data types for csv
edgelist = pd.read_csv("/Users/akshaykulkarni/Downloads/twitter/edgelist", 
                       sep = "\t", names=["source","target"], dtype={'source': 'int', 'target': 'int'})

# Sampling the graph with rudimentary Max_Filter (can look into more elegant graph smapling methods)
e_filter = edgelist.query('`source` <= 100 & `target` <= 100')
e_filter = e_filter.sort_values(by=['source', 'target'])
e_filter

In [ ]:
# Preloaded demo karate-community graph
g = nx.karate_club_graph()
fig1, ax1 = plt.subplots(1, 1, figsize=(8, 6));
nx.draw_networkx(g, ax=ax1)

# Twitter graph
G = nx.from_pandas_edgelist(e_filter, "source", "target")
fig, ax = plt.subplots(1, 1, figsize=(10, 10));
nx.draw_networkx(G, ax=ax)

#### Ironically NetworkX has a 2d spectral graphing function which Draws the graph G with a spectral 2D layout.

Using the unnormalized Laplacian, the layout shows possible clusters of nodes which are an approximation of the ratio cut. The positions are the entries of the second and third eigenvectors corresponding to the ascending eigenvalues starting from the second one.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10));
print("Sampled Spectral graph ")
nx.draw_spectral(G, ax=ax)


##  Creating JSON dumps of graph object data for JS stuff  

In [ ]:
# defining custom exporting function
def exportToJSON(Graph, filename):
    nodes = [{'name': str(i)}
             for i in Graph.nodes()]
    links = [{'source': u[0], 'target': u[1]}
             for u in Graph.edges()]
    with open(filename+'.json', 'w') as f:
        json.dump({'nodes': nodes, 'links': links},
                  f, indent=4,)

In [ ]:
exportToJSON(G,"graph")
# exportToJSON(g,"graph1")

In [ ]:
# turns out networkx has fuctionality but the above func can be used to export minimal data 
# Converting graph objects into JSON to be read by D3
from networkx.readwrite import json_graph
data = json_graph.node_link_data(g)
with open('karate.json', 'w') as f:
    json.dump(data, f, indent=4)
    

from networkx.readwrite import json_graph
data = json_graph.node_link_data(G)
with open('HTT.json', 'w') as f:
    json.dump(data, f, indent=4)

# D3 visualisation  stuff

In [ ]:
# D3 visualisations can be rendered in an ipynb notebook by injecting JS code in html cells ...
# for some god damn fucking reason its working just fine with the example karate graph 
# but does not render our HTT twitter graph I HAVE NO IDEA WHY ... i've been wrestling with d3 since Atlanta .. 
# though i have managed to view our graphs in d3 in other code cells just not in our notebook here.


# run html cell then run the next JS cell the graph will come as output of the html cell


In [ ]:
%%html
<div id="d3-container"> </div>

<style>

.node {stroke: #fff; stroke-width: 1.5px;}
.link {stroke: #999; stroke-opacity: .6;}

</style>

In [ ]:
%%html
<!DOCTYPE html>
<meta charset="utf-8">
<style>

.links line {
  stroke: #999;
  stroke-opacity: 0.6;
}

.nodes circle {
  stroke: black	;
  stroke-width: 0px;
}

</style>
<svg width="800" height="500"></svg>
<script src="https://d3js.org/d3.v4.min.js"></script>
<script>

//create somewhere to put the force directed graph
var svg = d3.select("svg"),
    width = +svg.attr("width"),
    height = +svg.attr("height");
    
var radius = 5; 

var nodes_data =  [
    {"name": "Lillian", "sex": "F"},
    {"name": "Gordon", "sex": "M"},
    {"name": "Sylvester", "sex": "M"},
    {"name": "Mary", "sex": "F"},
    {"name": "Helen", "sex": "F"},
    {"name": "Jamie", "sex": "M"},
    {"name": "Jessie", "sex": "F"},
    {"name": "Ashton", "sex": "M"},
    {"name": "Duncan", "sex": "M"},
    {"name": "Evette", "sex": "F"},
    {"name": "Mauer", "sex": "M"},
    {"name": "Fray", "sex": "F"},
    {"name": "Duke", "sex": "M"},
    {"name": "Baron", "sex": "M"},
    {"name": "Infante", "sex": "M"},
    {"name": "Percy", "sex": "M"},
    {"name": "Cynthia", "sex": "F"},
    {"name": "Feyton", "sex": "M"},
    {"name": "Lesley", "sex": "F"},
    {"name": "Yvette", "sex": "F"},
    {"name": "Maria", "sex": "F"},
    {"name": "Lexy", "sex": "F"},
    {"name": "Peter", "sex": "M"},
    {"name": "Ashley", "sex": "F"},
    {"name": "Finkler", "sex": "M"},
    {"name": "Damo", "sex": "M"},
    {"name": "Imogen", "sex": "F"}
    ]


var links_data = [
	{"source": "Sylvester", "target": "Gordon", "type":"A" },
    {"source": "Sylvester", "target": "Lillian", "type":"A" },
    {"source": "Sylvester", "target": "Mary", "type":"A"},
    {"source": "Sylvester", "target": "Jamie", "type":"A"},
    {"source": "Sylvester", "target": "Jessie", "type":"A"},
    {"source": "Sylvester", "target": "Helen", "type":"A"},
    {"source": "Helen", "target": "Gordon", "type":"A"},
    {"source": "Mary", "target": "Lillian", "type":"A"},
    {"source": "Ashton", "target": "Mary", "type":"A"},
    {"source": "Duncan", "target": "Jamie", "type":"A"},
    {"source": "Gordon", "target": "Jessie", "type":"A"},
    {"source": "Sylvester", "target": "Fray", "type":"E"},
    {"source": "Fray", "target": "Mauer", "type":"A"},
    {"source": "Fray", "target": "Cynthia", "type":"A"},
    {"source": "Fray", "target": "Percy", "type":"A"},
    {"source": "Percy", "target": "Cynthia", "type":"A"},
    {"source": "Infante", "target": "Duke", "type":"A"},
    {"source": "Duke", "target": "Gordon", "type":"A"},
    {"source": "Duke", "target": "Sylvester", "type":"A"},
    {"source": "Baron", "target": "Duke", "type":"A"},
    {"source": "Baron", "target": "Sylvester", "type":"E"},
    {"source": "Evette", "target": "Sylvester", "type":"E"},
    {"source": "Cynthia", "target": "Sylvester", "type":"E"},
    {"source": "Cynthia", "target": "Jamie", "type":"E"},
    {"source": "Mauer", "target": "Jessie", "type":"E"},
    {"source": "Duke", "target": "Lexy", "type":"A"},
    {"source": "Feyton", "target": "Lexy", "type":"A"},
    {"source": "Maria", "target": "Feyton", "type":"A"},
    {"source": "Baron", "target": "Yvette", "type":"E"},
    {"source": "Evette", "target": "Maria", "type":"E"},
    {"source": "Cynthia", "target": "Yvette", "type":"E"},
    {"source": "Maria", "target": "Jamie", "type":"E"},
    {"source": "Maria", "target": "Lesley", "type":"E"},
    {"source": "Ashley", "target": "Damo", "type":"A"},
    {"source": "Damo", "target": "Lexy", "type":"A"},
    {"source": "Maria", "target": "Feyton", "type":"A"},
    {"source": "Finkler", "target": "Ashley", "type":"E"},
    {"source": "Sylvester", "target": "Maria", "type":"E"},
    {"source": "Peter", "target": "Finkler", "type":"E"},
    {"source": "Ashley", "target": "Gordon", "type":"E"},
    {"source": "Maria", "target": "Imogen", "type":"E"}
    
]


//set up the simulation and add forces  
var simulation = d3.forceSimulation()
					.nodes(nodes_data);
                              
var link_force =  d3.forceLink(links_data)
                        .id(function(d) { return d.name; });            
         
var charge_force = d3.forceManyBody()
    .strength(-100); 
    
var center_force = d3.forceCenter(width / 2, height / 2);  
                      
simulation
    .force("charge_force", charge_force)
    .force("center_force", center_force)
    .force("links",link_force);

        
//add tick instructions: 
simulation.on("tick", tickActions );

//add encompassing group for the zoom 
var g = svg.append("g")
    .attr("class", "everything");

//draw lines for the links 
var link = g.append("g")
      .attr("class", "links")
    .selectAll("line")
    .data(links_data)
    .enter().append("line")
      .attr("stroke-width", 2)
      .style("stroke", linkColour);        

//draw circles for the nodes 
var node = g.append("g")
        .attr("class", "nodes") 
        .selectAll("circle")
        .data(nodes_data)
        .enter()
        .append("circle")
        .attr("r", radius)
        .attr("fill", circleColour);
 
 
//add drag capabilities  
var drag_handler = d3.drag()
	.on("start", drag_start)
	.on("drag", drag_drag)
	.on("end", drag_end);	
	
drag_handler(node);


//add zoom capabilities 
var zoom_handler = d3.zoom()
    .on("zoom", zoom_actions);

zoom_handler(svg);     

/** Functions **/

//Function to choose what color circle we have
//Let's return blue for males and red for females
function circleColour(d){
	if(d.sex =="M"){
		return "blue";
	} else {
		return "pink";
	}
}

//Function to choose the line colour and thickness 
//If the link type is "A" return green 
//If the link type is "E" return red 
function linkColour(d){
	if(d.type == "A"){
		return "green";
	} else {
		return "red";
	}
}

//Drag functions 
//d is the node 
function drag_start(d) {
 if (!d3.event.active) simulation.alphaTarget(0.3).restart();
    d.fx = d.x;
    d.fy = d.y;
}

//make sure you can't drag the circle outside the box
function drag_drag(d) {
  d.fx = d3.event.x;
  d.fy = d3.event.y;
}

function drag_end(d) {
  if (!d3.event.active) simulation.alphaTarget(0);
  d.fx = null;
  d.fy = null;
}

//Zoom functions 
function zoom_actions(){
    g.attr("transform", d3.event.transform)
}

function tickActions() {
    //update circle positions each tick of the simulation 
       node
        .attr("cx", function(d) { return d.x; })
        .attr("cy", function(d) { return d.y; });
        
    //update link positions 
    link
        .attr("x1", function(d) { return d.source.x; })
        .attr("y1", function(d) { return d.source.y; })
        .attr("x2", function(d) { return d.target.x; })
        .attr("y2", function(d) { return d.target.y; });
} 
</script>

In [ ]:
%%javascript
require.config({paths: {d3: "https://d3js.org/d3.v3.min"}});
require(["d3"], function(d3) {
    var width = 600,
        height = 600;

    var color = d3.scale.category10();

    // Creating a force-directed dynamic graph layout.
    var force = d3.layout.force()
        .charge(-500)
        .linkDistance(50)
        .size([width, height]);

    var svg = d3.select("#d3-container").select("svg")
    if (svg.empty()) {
        svg = d3.select("#d3-container").append("svg")
                    .attr("width", width)
                    .attr("height", height);
    }

    // load the JSON network file.
    d3.json("karate.json", function(error, tgraph) {
        // Within this block, the network has been loaded and stored
        // load the nodes and links into the force-directed
        // graph and initialise the dynamics.
        force.nodes(tgraph.nodes)
            .links(tgraph.links)
            .start();

        // We create a <line> SVG element for each link
        // in the graph.
        var link = svg.selectAll(".link")
            .data(tgraph.links)
            .enter().append("line")
            .attr("class", "link");

        // create a <circle> SVG element for each node
        // in the graph, and we specify a few attributes.
        var node = svg.selectAll(".node")
            .data(tgraph.nodes)
            .enter().append("circle")
            .attr("class", "node")
            .attr("r", 5)  // radius
            .call(force.drag);

        // The label each node its node number from the networkx graph.
        node.append("title")
            .text(function(d) { return d.id; });

        // bind the positions of the SVG elements
        // to the positions of the dynamic force-directed graph,
        // at each time step.
        force.on("tick", function() {
            link.attr("x1", function(d) { return d.source.x; })
                .attr("y1", function(d) { return d.source.y; })
                .attr("x2", function(d) { return d.target.x; })
                .attr("y2", function(d) { return d.target.y; });

            node.attr("cx", function(d) { return d.x; })
                .attr("cy", function(d) { return d.y; });
        });
    });
});

-------------

# COVID-19 Mined Dataset

In [ ]:
import json
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from networkx.readwrite import json_graph

covid_raw = pd.read_csv("Data/Hashtag_Raw_Data/COVID19_hashtags7.csv").drop(["Unnamed: 0"], axis =1)
display(covid_raw)

print(covid_raw.nunique())

covid_raw['hashtag_id'] = covid_raw.groupby(['hashtag_x']).ngroup()
covid_raw.columns = ["tweet_id", "hashtag_name","tweet_hashtag_count", "hashtag_id"]

display(covid_raw)
covid_nodelist = covid_raw.drop_duplicates(['hashtag_name','hashtag_id'])[['hashtag_name','hashtag_id']].copy()

covid_nodelist = covid_nodelist.sort_values(by=['hashtag_id'])

covid_nodelist.to_csv("COVID19_NodeList.csv", index= False)

covid_nodelist

In [ ]:
from itertools import combinations

covid_edgelist_src = covid_raw[["tweet_id", "hashtag_id"]]

list_of_edges = []

# Createing edges
for i, group in covid_edgelist_src.groupby(['tweet_id'])['hashtag_id']:
    # generate all combinations without replacement 
    # from the group of similar column pairs
    for u, v in combinations(group, 2):           
        list_of_edges.append(sorted([u, v]))
        


covid_edgelist = pd.DataFrame(list_of_edges, columns=["Source","Target"])


covid_edgelist = covid_edgelist.sort_values(by=['Source',"Target"])

covid_edgelist = covid_edgelist.groupby(covid_edgelist.columns.tolist()).size().reset_index().\
    rename(columns={0:'Edge_Freq'})

print(covid_edgelist.shape)
covid_edgelist

In [ ]:
# Twitter graph
# temp = covid_edgelist.query('`Source` <= 50 & `Target` <= 50')
# display(temp)
C = nx.from_pandas_edgelist(covid_edgelist, "Source", "Target",['Edge_Freq'])
fig, ax = plt.subplots(1, 1, figsize=(20, 20));
nx.draw(C, ax=ax)

In [ ]:
test = covid_nodelist.set_index('hashtag_id') 
test['Communities'] = np.arange(3509)

test

In [ ]:
attrs = test.to_dict('index')
attrs
nx.set_node_attributes(C, attrs)

In [ ]:
C.nodes[1299]

In [ ]:
import networkx as nx
def connected_component_subgraphs(G):
    for c in nx.connected_components(G):
        yield G.subgraph(c).copy()
   

G.subgraph(c).copy() for c in connected_components(G)

connected_nodes = max(connected_component_subgraphs(C), key=len)

print(len(C.nodes))
print(len(connected_nodes.nodes))

In [ ]:
connected_nodes.nodes[1299]['hashtag_name']

In [ ]:
#Saving

covid_nodelist.to_csv("COVID19_NodeList.csv", index= False)
covid_edgelist.to_csv("COVID19_EdgeList.csv", index= False)

## Experimental Stuff

Libs
- pyvis ( main opt)
- graphistry ( high capacity opt )
- d3g (bs)

In [ ]:
!pwd

import os
os.chdir('..')
!pwd

In [ ]:
covid_raw = pd.read_csv("Data/Hashtag_Raw_Data/COVID19_hashtags7.csv").drop(["Unnamed: 0"], axis =1)
display(covid_raw)

print(covid_raw.nunique())

covid_raw['hashtag_id'] = covid_raw.groupby(['hashtag_x']).ngroup()
covid_raw.columns = ["tweet_id", "hashtag_name","tweet_hashtag_count", "hashtag_id"]

display(covid_raw)
covid_nodelist = covid_raw.drop_duplicates(['hashtag_name','hashtag_id'])[['hashtag_name','hashtag_id']].copy()

covid_nodelist = covid_nodelist.sort_values(by=['hashtag_id'])

covid_nodelist.to_csv("COVID19_NodeList.csv", index= False)

covid_nodelist

In [ ]:
from itertools import combinations

covid_edgelist_src = covid_raw[["tweet_id", "hashtag_id"]]

test = covid_raw.query('`hashtag_id` <= 5')
display(test)

list_of_edges = []

# Createing edges
for i, group in test.groupby(['tweet_id'])['hashtag_id']:
    # generate all combinations without replacement 
    # from the group of similar column pairs
    for u, v in combinations(group, 2):           
        list_of_edges.append(sorted([u, v]))
        


covid_edgelist = pd.DataFrame(list_of_edges, columns=["Source","Target"])


covid_edgelist = covid_edgelist.sort_values(by=['Source',"Target"])



print(covid_edgelist.shape)
covid_edgelist

In [ ]:
from itertools import combinations

covid_edgelist_src = covid_raw[["tweet_id", "hashtag_id"]]

test = covid_raw.query('`hashtag_id` <= 5')
display(test)

list_of_edges = []

# Createing edges
for i, group in covid_edgelist_src.groupby(['tweet_id'])['hashtag_id']:
    # generate all combinations without replacement 
    # from the group of similar column pairs
    for u, v in combinations(group, 2):           
        list_of_edges.append([u, v])
        


covid_edgelist = pd.DataFrame(list_of_edges, columns=["Source","Target"])


covid_edgelist = covid_edgelist.sort_values(by=['Source',"Target"])



print(covid_edgelist.shape)
covid_edgelist.drop_duplicates(['Source','Target'])

In [ ]:
island = pd.read_csv("/Users/akshaykulkarni/Downloads/connected_edges.csv")
C = nx.from_pandas_edgelist(island, "Source", "Target")
# fig, ax = plt.subplots(1, 1, figsize=(20, 20))
# nx.draw_kamada_kawai(C, ax=ax)

In [ ]:
from networkx.readwrite import json_graph
data = json_graph.node_link_data(C)
with open('island.json', 'w') as f:
    json.dump(data, f, indent=4)

In [ ]:
from pyvis.network import Network as pyNet
import networkx as nx
G = pyNet(notebook = True)
G.show_buttons(filter_=['physics'])
G.from_nx(C)
G.show('island2.html')

In [ ]:
island[island['Source'] == 1299]

In [1]:
import pandas as pd
nodes = pd.read_csv('data/Island/connected_nodes.csv')

In [2]:
nodes[nodes['hashtag_id'].isin([2313,2810,2824,2865])]

,hashtag_name,hashtag_id
1164,education,2313
1447,ncssethebest,2810
1459,newteacheradvice,2824
1480,oldteacheradvice,2865


In [16]:
 x= nodes[nodes['hashtag_id'].isin([17, 56, 112, 121, 129, 137, 150, 151, 173, 176, 198, 202, 263, 463, 559, 620, 644, 645, 646, 650, 658, 731, 774, 841, 873, 919, 923, 924, 981, 988, 990, 1050, 1204, 1207, 1214, 1256, 1259, 1290, 1299, 1311, 1355, 1437, 1449, 1452, 1478, 1485, 1499, 1509, 1512, 1534, 1537, 1543, 1544, 1555, 1562, 1565, 1572, 1594, 1630, 1645, 1647, 1652, 1688, 1689, 1696, 1720, 1792, 1798, 1812, 1848, 1950, 1975, 1976, 1989, 1997, 2064, 2083, 2090, 2123, 2129, 2135, 2164, 2239, 2292, 2309, 2425, 2474, 2507, 2539, 2548, 2551, 2598, 2602, 2657, 2791, 2838, 2882, 2885, 2905, 2906, 2907, 2913, 2990, 3028, 3038, 3087, 3121, 3144, 3186, 3195, 3279, 3302, 3315, 3367, 3380, 3434, 3461, 3474, 3477, 3483, 3487]
)]

In [22]:
pd.set_option('display.max_rows', 500)


x.to_csv("1299.csv")

In [7]:
comm_35 = [735,1111,1563,1889,1980,1985,2052,2099,2131,2148,2380,2381,2389,2515,2519,2608,2877,2988,3054,3196,3234,3326,3339,3455]
nodes[nodes['hashtag_id'].isin(comm_35)]


,hashtag_name,hashtag_id
351,HowCanIHelp,735
535,Nature,1111
768,StrongerTogether,1563
943,academics,1889
986,begenerous,1980
989,bekind,1985
1031,businessnews,2052
1056,charity,2099
1071,closingequitygaps,2131
1078,community,2148


In [23]:
GN_41= [621, 1292, 1898, 2422, 2648, 2649, 3382]
nodes[nodes['hashtag_id'].isin(GN_41)]


,hashtag_name,hashtag_id
295,GPU,621
625,QLOCdragons,1292
946,agoraphobia,1898
1225,gamers,2422
1356,korona,2648
1357,koronavirus,2649
1757,videogame,3382


In [ ]:
GN_40= [334, 1604, 2737, 2738, 2802, 2814, 2815, 3101]
nodes[nodes['hashtag_id'].isin(GN_40)]


In [ ]:
nodes[nodes['hashtag_id'].isin([1265,1938,2287,2288,2327,2550,2682,3427])]

In [4]:
comm_35 = [735,1111,1563,1889,1980,1985,2052,2099,2131,2148,2380,2381,2389,2515,2519,2608,2877,2988,3054,3196,3234,3326,3339,3455]
nodes[nodes['hashtag_id'].isin(comm_35)]


,hashtag_name,hashtag_id
351,HowCanIHelp,735
535,Nature,1111
768,StrongerTogether,1563
943,academics,1889
986,begenerous,1980
989,bekind,1985
1031,businessnews,2052
1056,charity,2099
1071,closingequitygaps,2131
1078,community,2148


In [67]:
CNM_37 =[777, 2268, 1627, 2378, 2849, 2445, 2743, 2848, 2938, 3286]
GN_39= [777, 1627, 2268, 2378, 2445, 2743, 2848, 2849, 2938, 3286]



nodes[nodes['hashtag_id'].isin(GN_39)]




MED_DF = pd.concat([nodes[nodes['hashtag_id'].isin(CNM_37)],nodes[nodes['hashtag_id'].isin(GN_39)]],axis =1)
MED_DF.to_csv('CNM37vsGN39.csv')

In [68]:
CNM_9 =[21, 2265, 2605, 3069, 496, 1360, 1372, 1385, 649, 1340, 1524, 1638, 2248, 2347, 2541, 2604, 2748, 2953, 1467, 2127, 2346]
GN_33 =[496, 649, 818, 1310, 1340, 1348, 1360, 1372, 1385, 1467, 1524, 1638, 1838, 2127, 2248, 2541, 2604, 2748, 2953]


# display(nodes[nodes['hashtag_id'].isin(CNM_9)],nodes[nodes['hashtag_id'].isin(GN_33)])

MED_DF = pd.concat([nodes[nodes['hashtag_id'].isin(CNM_9)],nodes[nodes['hashtag_id'].isin(GN_33)]],axis =1)

MED_DF.to_csv('CNM9vsGN33.csv')
# MED_DF.to_csv('_comparison.csv')


In [ ]:
CNM_20 = [378, 1435, 1440, 1731]


In [5]:
GN_40= [334, 1604, 2737, 2738, 2802, 2814, 2815, 3101]
nodes[nodes['hashtag_id'].isin(GN_40)]


,hashtag_name,hashtag_id
153,ClickOnCare,334
791,TaiwanCanHelp,1604
1406,masks,2737
1407,masques,2738
1444,namaste,2802
1451,netflix,2814
1452,netflixandchill,2815
1603,sanitizer,3101


In [ ]:
import json

with open('GN_SY_Test_1.json') as f:
    data = json.load(f)

print(data)


In [37]:
gn_comms = {}
for i in data['nodes']:
    gn_comms[i['id']] = i['CNM_Label']

In [38]:
gn_comms

{1: 15,
 2228: 15,
 2828: 15,
 3175: 15,
 6: 27,
 2589: 27,
 3098: 27,
 7: 2,
 2092: 2,
 2448: 2,
 10: 9,
 74: 9,
 196: 9,
 319: 2,
 399: 9,
 1048: 9,
 1088: 20,
 1338: 9,
 1719: 9,
 1768: 9,
 1785: 9,
 1964: 9,
 2157: 7,
 2301: 9,
 2801: 9,
 3013: 9,
 3295: 9,
 12: 20,
 2491: 20,
 2622: 20,
 3409: 20,
 13: 16,
 614: 16,
 15: 11,
 2306: 11,
 2411: 11,
 2412: 11,
 17: 1,
 1299: 1,
 1555: 1,
 1792: 1,
 3461: 1,
 18: 8,
 1258: 8,
 2522: 6,
 2546: 8,
 2898: 8,
 3274: 14,
 21: 2,
 2265: 2,
 2605: 2,
 3069: 2,
 22: 12,
 1544: 1,
 2706: 12,
 26: 36,
 3320: 36,
 27: 31,
 87: 31,
 1285: 31,
 2726: 31,
 2937: 31,
 28: 18,
 737: 18,
 1855: 18,
 2294: 18,
 29: 8,
 3096: 8,
 3233: 8,
 38: 12,
 227: 26,
 713: 12,
 42: 9,
 417: 9,
 1356: 9,
 1999: 9,
 49: 7,
 64: 7,
 1483: 7,
 56: 1,
 129: 1,
 1534: 1,
 57: 4,
 69: 4,
 60: 7,
 62: 7,
 1236: 7,
 2892: 7,
 3350: 7,
 61: 7,
 577: 7,
 63: 26,
 845: 26,
 846: 38,
 1557: 26,
 67: 2,
 802: 2,
 813: 2,
 1505: 23,
 68: 12,
 2695: 12,
 117: 4,
 445: 4,
 697: 4

In [39]:
for e in data['links']:
    if gn_comms[e['source']] == gn_comms[e['target']]:
        e['color'] = gn_comms[e['source']]
    else:
        e['color'] = 0

In [ ]:
data

In [20]:
with open('GirvanNewman.json', 'w') as json_file:
    json.dump(data, json_file)

In [41]:
from networkx.readwrite import json_graph

GN = json_graph.node_link_graph(data)

In [42]:
degs = {}
for i in GN.nodes:
    degs[i] = GN.degree[i]

In [43]:
import networkx as nx
nx.set_node_attributes(GN, degs, 'Degree')

In [44]:
GN_data = json_graph.node_link_data(GN)

In [45]:
with open('GirvanNewman.json', 'w') as json_file:
    json.dump(GN_data, json_file)